<a href="https://colab.research.google.com/github/bonioloff/note_pyspark/blob/main/PySpark_Note.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Explanation on how Spark works:
- https://stackoverflow.com/questions/32621990/what-are-workers-executors-cores-in-spark-standalone-cluster

In [ ]:
# install the pyspark package
!pip install pyspark

In [2]:
from pyspark import SparkConf, SparkContext

In [3]:
# Initialize spark context
conf = SparkConf().setMaster("local").setAppName("Spark App")
sc = SparkContext(conf=conf)

# Resilient Distributed Dataset (RDD) Interface

RDD can be used to store all types of data source:
- Manual entry
- textFile from local file, s3, hdfs, etc.
- Hive
- JDBC
- Cassandra
- HBase
- ElasticSearch
- JSON, etc

In [8]:
# Manual way to create RDD
rdd = sc.parallelize([1,2,3,4])

In [13]:
# Create RDD from text file
rdd = sc.textFile("sample_data/california_housing_test.csv")

In [14]:
# Show the first 5 rows
rdd.take(5)
# rdd.collect()

['"longitude","latitude","housing_median_age","total_rooms","total_bedrooms","population","households","median_income","median_house_value"',
 '-122.050000,37.370000,27.000000,3885.000000,661.000000,1537.000000,606.000000,6.608500,344700.000000',
 '-118.300000,34.260000,43.000000,1510.000000,310.000000,809.000000,277.000000,3.599000,176500.000000',
 '-117.810000,33.780000,27.000000,3589.000000,507.000000,1484.000000,495.000000,5.793400,270500.000000',
 '-118.360000,33.820000,28.000000,67.000000,15.000000,49.000000,11.000000,6.135900,330000.000000']

In [32]:
# flatMap()
rdd.flatMap(lambda x: x.split(",")).take(10)

['"longitude"',
 '"latitude"',
 '"housing_median_age"',
 '"total_rooms"',
 '"total_bedrooms"',
 '"population"',
 '"households"',
 '"median_income"',
 '"median_house_value"',
 '-122.050000']

## Basic Transformations with RDD
These are the basic operations in rdd: 
- map, 
- flatmap, 
- filter, 
- distinct, 
- sample, 
- union, 
- intersection, 
- subtract, 
- cartesian

_bykey_:
- reduceByKey()
- groupByKey()
- sortByKey()
- keys()
- values()

### Map
This will do transformation on the data row by row.

In [23]:
# map()
income_vs_homevalue = rdd.map(lambda x: x.split(",")).map(lambda x: (x[-2], x[-1]))
income_vs_homevalue.take(5)

[('"median_income"', '"median_house_value"'),
 ('6.608500', '344700.000000'),
 ('3.599000', '176500.000000'),
 ('5.793400', '270500.000000'),
 ('6.135900', '330000.000000')]

In [22]:
# reduceByKey()
income_vs_homevalue.reduceByKey(lambda x, y: x + y).take(5)

[('"median_income"', '"median_house_value"'),
 ('6.608500', '344700.000000'),
 ('3.599000', '176500.000000'),
 ('5.793400', '270500.000000'),
 ('6.135900', '330000.000000106300.000000225000.000000')]

In [24]:
# keys()
income_vs_homevalue.keys().take(5)

['"median_income"', '6.608500', '3.599000', '5.793400', '6.135900']

In [25]:
# values()
income_vs_homevalue.values().take(5)

['"median_house_value"',
 '344700.000000',
 '176500.000000',
 '270500.000000',
 '330000.000000']

In [34]:
# sortByKey()
income_vs_homevalue.sortByKey().take(5)

[('"median_income"', '"median_house_value"'),
 ('0.499900', '500001.000000'),
 ('0.536000', '162500.000000'),
 ('0.536000', '275000.000000'),
 ('0.536000', '87500.000000')]

# SparkSQL: DataFrame (DF) / Datasets

Unlike RDD, DataFrame has some benefits:
- Contains row objects
- Can run sql
- Can have schema
- Can communicate JDBC and ODBC and Tableau

Instead using SparkContext, DF lives in SparkSession.

__Datasets__ are typed Dataframe, it's used in typed programming language like Scala and Java, but in Python that is untyped we can use DataFrame.

Typed means we have to specifically tell what is the data type of the columns.


In [43]:
from pyspark.sql import SparkSession, Row

In [50]:
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [55]:
inputData = spark.read.csv("sample_data/california_housing_test.csv", header=True, inferSchema=True)

In [59]:
inputData.show(3)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|  -122.05|   37.37|              27.0|     3885.0|         661.0|    1537.0|     606.0|       6.6085|          344700.0|
|   -118.3|   34.26|              43.0|     1510.0|         310.0|     809.0|     277.0|        3.599|          176500.0|
|  -117.81|   33.78|              27.0|     3589.0|         507.0|    1484.0|     495.0|       5.7934|          270500.0|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
only showing top 3 rows



In [57]:
inputData.select("longitude").show(5)

+---------+
|longitude|
+---------+
|  -122.05|
|   -118.3|
|  -117.81|
|  -118.36|
|  -119.67|
+---------+
only showing top 5 rows



In [58]:
inputData.filter(inputData.total_rooms > 1000).show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|  -122.05|   37.37|              27.0|     3885.0|         661.0|    1537.0|     606.0|       6.6085|          344700.0|
|   -118.3|   34.26|              43.0|     1510.0|         310.0|     809.0|     277.0|        3.599|          176500.0|
|  -117.81|   33.78|              27.0|     3589.0|         507.0|    1484.0|     495.0|       5.7934|          270500.0|
|  -119.67|   36.33|              19.0|     1241.0|         244.0|     850.0|     237.0|       2.9375|           81700.0|
|  -119.56|   36.51|              37.0|     1018.0|         213.0|     663.0|     204.0|       1.6635|           67000.0|
+---------+--------+----

In [60]:
inputData.groupBy("households").count().show(5)

+----------+-----+
|households|count|
+----------+-----+
|     305.0|    4|
|     558.0|    2|
|     496.0|    1|
|     596.0|    5|
|     299.0|    4|
+----------+-----+
only showing top 5 rows

